# Harassment and Newcomer Retention

In this notebook we investigate how receiving harassment correlates with newcomer activity and retention. For the purposes of this study, our measures of harassment are classifiers over individual discussion comments for personal attacks, aggression and toxicity. These classifiers were developed [in previous work](https://arxiv.org/abs/1610.08914). We will investigate the relationship between harassment and newcomer retention through running regression models that use a measures of editing activity and harassment in a first time span (t1) as independent variables and a measure of editing activity in a subsequent time span (t2) as the dependent variable.

In the following analyses, t1 is the first month after registering and t2 is the second month after registering, but this could be modified in further analysis. Our initial set of examples consists of all (20k) newcomers to English Wikipedia from 2001-2015 that received a personal attack or an aggressive or toxic comment on their user page by another user in t1 and a random sample of 100k newcomers from 2001-2015. We the reduced the set of examples to those newcomers who made at least one edit in t1. 

In [53]:
% matplotlib inline
import pandas as pd
from dateutil.relativedelta import relativedelta
import statsmodels.formula.api as sm
import requests

### Regression Modeling


In [54]:
#Features computes in ./Harassment and Newcomer Retention Data Munging.ipynb
df_reg = pd.read_csv("../../data/retention/newcomer_sample_features.csv")

In [55]:
df_reg.shape

(111290, 24)

In [56]:
df_newcomer_sample = pd.read_csv("../../data/retention/newcomer_sample.csv")
df_reg = df_reg.merge(df_newcomer_sample, on = "user_text", how = "inner")

In [57]:
df_reg.shape

(111290, 26)

In [58]:
df_reg['sample'].value_counts()

random      92469
attacked    18821
Name: sample, dtype: int64

In [59]:
df_reg['t1_harassment_received'].value_counts()

0    108887
1      2403
Name: t1_harassment_received, dtype: int64

In [60]:
df_reg['has_gender'].value_counts()

0    105896
1      5394
Name: has_gender, dtype: int64

In [61]:
df_reg['t1_active'].value_counts()

1    111290
Name: t1_active, dtype: int64

In [62]:
df_reg = pd.concat(
    [df_reg.query('t1_harassment_received == 1'), df_reg.query("sample == 'random'").sample(50000, random_state = 12)]
).drop_duplicates(subset = "user_text")

In [63]:
df_reg['sample'].value_counts()

random      50014
attacked     2375
Name: sample, dtype: int64

In [64]:
df_reg['t1_harassment_received'].value_counts()

0    49986
1     2403
Name: t1_harassment_received, dtype: int64

In [65]:
df_reg.shape

(52389, 26)

In [66]:
column_map = {
        't1_num_days_active': 'm1_days_active',
        't2_num_days_active' : 'm2_days_active',
        't1_harassment_received': 'm1_received_harassment',
        't1_harassment_made': 'm1_made_harassment',
        't1_fraction_ns0_deleted': 'm1_fraction_ns0_deleted',
        't1_fraction_ns0_reverted': 'm1_fraction_ns0_reverted',
        't1_num_warnings_recieved': 'm1_warnings',

        }
        
df_reg = df_reg.rename(columns=column_map)

In [38]:
from io import StringIO
import math
import pandas as pd


def reformat_ci(s):
    ci = s.strip().split()
    ci = (round(float(ci[0]), 1), round(float(ci[1]), 1))
    return "[%.1f, %.1f]" % ci

def regress(df, f, family = 'linear'):
    if family == 'linear':
        results = sm.ols(formula=f, data=df).fit()

    elif family == 'logistic':
        results = sm.logit(formula=f, data=df).fit(disp=0)
    else:
        return
        
    results = pd.read_csv(StringIO(results.summary().tables[1].as_csv()))
    
    if family == 'linear':
        column_map = {
            results.columns[0]: "",
            '   coef   ' : 'coef',
           'P>|t| ': "p-val",
            '    t    ': "z-stat",
           ' [95.0% Conf. Int.]': "95% CI"
        }

    elif family == 'logistic':
        column_map = {
            results.columns[0]: "",
            '   coef   ' : 'coef',
           'P>|z| ': "p-val",
            '    z    ': "z-stat",
           ' [95.0% Conf. Int.]': "95% CI"
        }
    else:
        return
        
        
    results = results.rename(columns=column_map)
    results.index = results[""]
    del results[""]
    results = results[['coef', "z-stat", "p-val", "95% CI"]]
    results['coef'] = results['coef'].apply(lambda x: round(float(x), 2))
    results['z-stat'] = results['z-stat'].apply(lambda x: round(float(x), 1))
    results['p-val'] = results['p-val'].apply(lambda x: round(float(x), 3))
    results['95% CI'] = results['95% CI'].apply(reformat_ci)
    header = """
\\begin{table}[h]
\\begin{center}
    """
    footer = """
\\end{center}
\\caption{%s}
\\label{tab:}
\\end{table}
    """
    f = f.replace("_", "\_").replace("~", "\\texttildelow\\")
    latex = header + results.to_latex() + footer % f
    print(latex)
    return results
        
    
    

#### Linear Regression: Is harassment correlated with reduction in activity from t1 to t2?

Instead of running a logistic regression using an indicator for activity in t2 as our dependent variable, we will run a linear regression using the the number of days active in t2 as our dependent variable.

In [27]:
f ="m2_days_active ~ m1_received_harassment"
regress(df_reg, f)


\begin{table}[h]
\begin{center}
    \begin{tabular}{lrrrl}
\toprule
{} &  coef &  z-stat &  p-val &      95\% CI \\
\midrule
                       &       &         &        &             \\
Intercept              &  0.18 &    23.9 &    0.0 &  [0.2, 0.2] \\
m1\_received\_harassment &  1.86 &    53.2 &    0.0 &  [1.8, 1.9] \\
\bottomrule
\end{tabular}

\end{center}
\caption{m2\_days\_active \texttildelow\ m1\_received\_harassment}
\label{}
\end{table}
    


,coef,z-stat,p-val,95% CI
,,,,
Intercept,0.18,23.9,0.0,"[0.2, 0.2]"
m1_received_harassment,1.86,53.2,0.0,"[1.8, 1.9]"


We see a similar results as above. Without controlling for activity in t1, users who receive harassment have, on average, more active days in t2.

In [28]:
f= "m2_days_active ~ m1_days_active + m1_received_harassment"
regress(df_reg, f)


\begin{table}[h]
\begin{center}
    \begin{tabular}{lrrrl}
\toprule
{} &  coef &  z-stat &  p-val &        95\% CI \\
\midrule
                       &       &         &        &               \\
Intercept              & -0.57 &   -81.4 &    0.0 &  [-0.6, -0.6] \\
m1\_days\_active         &  0.49 &   191.0 &    0.0 &    [0.5, 0.5] \\
m1\_received\_harassment & -0.47 &   -16.1 &    0.0 &  [-0.5, -0.4] \\
\bottomrule
\end{tabular}

\end{center}
\caption{m2\_days\_active \texttildelow\ m1\_days\_active + m1\_received\_harassment}
\label{}
\end{table}
    


,coef,z-stat,p-val,95% CI
,,,,
Intercept,-0.57,-81.4,0.0,"[-0.6, -0.6]"
m1_days_active,0.49,191.0,0.0,"[0.5, 0.5]"
m1_received_harassment,-0.47,-16.1,0.0,"[-0.5, -0.4]"


However, when we control for the number of days a user is active in t1, we see that users who receive harassment have fewer active days in t2. The coefficient is significantly less than 0. Instead of using an indicator for whether the user received harassment, lets use the count of various types of harassment received (i.e personal attacks, aggression, toxicity)

#### How do gender, harassment and activity interact?

Gender in Wikipedia is not well defined. After registering, users have the ability to report their gender in their user preferences. The vast majority of users do not report their gender. This may be because reporting their gender is not important to them, they don't want to report a gender, or they simply are unaware of the feature. There is anecdotal evidence that users often report an incorrect gender. Overall, this means that we should expect users who report their gender to be different than the rest and we cannot be sure if reported genders are correct. Another caveat for the following analysis is that we do not know when the user reported their gender; they may have changed their user preference after our 2 month interval of interest.




Users who supply a gender are more likely to receive harassment! Lets see if this is different for males and females:

In [30]:
f="m1_received_harassment ~ is_female"
regress(df_reg.query("has_gender == 1"), f, family = 'logistic')


\begin{table}[h]
\begin{center}
    \begin{tabular}{lrrrl}
\toprule
{} &  coef &  z-stat &  p-val &        95\% CI \\
\midrule
          &       &         &        &               \\
Intercept & -1.90 &   -22.7 &  0.000 &  [-2.1, -1.7] \\
is\_female &  0.51 &     2.9 &  0.004 &    [0.2, 0.9] \\
\bottomrule
\end{tabular}

\end{center}
\caption{m1\_received\_harassment \texttildelow\ is\_female}
\label{}
\end{table}
    


,coef,z-stat,p-val,95% CI
,,,,
Intercept,-1.90,-22.7,0.000,"[-2.1, -1.7]"
is_female,0.51,2.9,0.004,"[0.2, 0.9]"


Females have an increased probability of receiving harassment in t1.

In [37]:
f="m2_days_active ~ m1_days_active + is_female"
regress(df_reg.query("has_gender == 1"), f)


\begin{table}[h]
\begin{center}
    \begin{tabular}{lrrrl}
\toprule
{} &  coef &  z-stat &  p-val &        95\% CI \\
\midrule
               &       &         &        &               \\
Intercept      & -0.87 &    -8.9 &  0.000 &  [-1.1, -0.7] \\
m1\_days\_active &  0.65 &    45.9 &  0.000 &    [0.6, 0.7] \\
is\_female      & -0.08 &    -0.4 &  0.714 &   [-0.5, 0.3] \\
\bottomrule
\end{tabular}

\end{center}
\caption{m2\_days\_active \texttildelow\ m1\_days\_active + is\_female}
\label{}
\end{table}
    


,coef,z-stat,p-val,95% CI
,,,,
Intercept,-0.87,-8.9,0.000,"[-1.1, -0.7]"
m1_days_active,0.65,45.9,0.000,"[0.6, 0.7]"
is_female,-0.08,-0.4,0.714,"[-0.5, 0.3]"


Females appear to have decreased activity in t2 even after controlling for activity in t1 compared to males.

In [46]:
f="m2_days_active ~ m1_days_active + m1_received_harassment + m1_received_harassment : is_female"
regress(df_reg.query("has_gender == 1"), f)


\begin{table}[h]
\begin{center}
    \begin{tabular}{lrrrl}
\toprule
{} &  coef &  z-stat &  p-val &        95\% CI \\
\midrule
                                 &       &         &        &               \\
Intercept                        & -0.86 &    -9.5 &  0.000 &  [-1.0, -0.7] \\
m1\_days\_active                   &  0.72 &    41.5 &  0.000 &    [0.7, 0.7] \\
m1\_received\_harassment           & -1.65 &    -5.7 &  0.000 &  [-2.2, -1.1] \\
m1\_received\_harassment:is\_female & -0.09 &    -0.2 &  0.854 &   [-1.0, 0.8] \\
\bottomrule
\end{tabular}

\end{center}
\caption{m2\_days\_active \texttildelow\ m1\_days\_active + m1\_received\_harassment + m1\_received\_harassment : is\_female}
\label{tab:}
\end{table}
    


,coef,z-stat,p-val,95% CI
,,,,
Intercept,-0.86,-9.5,0.000,"[-1.0, -0.7]"
m1_days_active,0.72,41.5,0.000,"[0.7, 0.7]"
m1_received_harassment,-1.65,-5.7,0.000,"[-2.2, -1.1]"
m1_received_harassment:is_female,-0.09,-0.2,0.854,"[-1.0, 0.8]"


It seems like users who supply a gender and receive harassment have even more strongly reduced activity in t2 compared to users who do not supply a gender and get harassed.

When restricting to users with a gender, we see that users with `t1_harassment_received` have reduced activity in t2, but the impact is not significantly different for males and females.

#### Addressing the bad newcomer confound

A serious potential confound in our analyses could be that the users who receive harassment are just bad faith newcomers or sock-puppets. They get attacked for their misbehavior and reduce their activity in t2 because they get blocked or because they never intended to stick around past their own attacks. To reduce this confound, we control for whether the user harassed anyone in t1 and for whether they received an user warning of any type:

In [41]:
f="m2_days_active ~ m1_days_active + m1_received_harassment +  m1_received_harassment : m1_made_harassment + m1_received_harassment : m1_warnings"
regress(df_reg, f)


\begin{table}[h]
\begin{center}
    \begin{tabular}{lrrrl}
\toprule
{} &  coef &  z-stat &  p-val &        95\% CI \\
\midrule
                                          &       &         &        &               \\
Intercept                                 & -0.57 &   -81.2 &  0.000 &  [-0.6, -0.6] \\
m1\_days\_active                            &  0.49 &   188.3 &  0.000 &    [0.5, 0.5] \\
m1\_received\_harassment                    & -0.39 &   -11.8 &  0.000 &  [-0.5, -0.3] \\
m1\_received\_harassment:m1\_made\_harassment & -0.10 &    -1.7 &  0.085 &   [-0.2, 0.0] \\
m1\_received\_harassment:m1\_warnings        & -0.15 &    -7.9 &  0.000 &  [-0.2, -0.1] \\
\bottomrule
\end{tabular}

\end{center}
\caption{m2\_days\_active \texttildelow\ m1\_days\_active + m1\_received\_harassment +  m1\_received\_harassment : m1\_made\_harassment + m1\_received\_harassment : m1\_warnings}
\label{tab:}
\end{table}
    


,coef,z-stat,p-val,95% CI
,,,,
Intercept,-0.57,-81.2,0.000,"[-0.6, -0.6]"
m1_days_active,0.49,188.3,0.000,"[0.5, 0.5]"
m1_received_harassment,-0.39,-11.8,0.000,"[-0.5, -0.3]"
m1_received_harassment:m1_made_harassment,-0.10,-1.7,0.085,"[-0.2, 0.0]"
m1_received_harassment:m1_warnings,-0.15,-7.9,0.000,"[-0.2, -0.1]"


Even users who receive harassment but did not harass anyone or receive a user warning show reduced activity in t2.

#### Harassment vs Previously identified newcomer socialization issues

[Halfak et al](https://www-users.cs.umn.edu/~halfak/publications/The_Rise_and_Decline/halfaker13rise-preprint.pdf) examine how user warnings and deletions and reverts correlate with newcomer retention. Here we add those features and see how they compare to measure of harassment.

In [67]:
f = "m2_days_active ~ m1_days_active + m1_received_harassment + m1_warnings +  m1_fraction_ns0_deleted + m1_fraction_ns0_reverted "
regress(df_reg.query("t1_num_ns0_edits > 0"), f)


\begin{table}[h]
\begin{center}
    \begin{tabular}{lrrrl}
\toprule
{} &  coef &  z-stat &  p-val &        95\% CI \\
\midrule
                         &       &         &        &               \\
Intercept                & -0.60 &   -49.8 &  0.000 &  [-0.6, -0.6] \\
m1\_days\_active           &  0.51 &   158.9 &  0.000 &    [0.5, 0.5] \\
m1\_received\_harassment   & -0.53 &   -13.0 &  0.000 &  [-0.6, -0.5] \\
m1\_warnings              & -0.12 &    -7.7 &  0.000 &  [-0.2, -0.1] \\
m1\_fraction\_ns0\_deleted  & -0.05 &    -1.1 &  0.272 &   [-0.1, 0.0] \\
m1\_fraction\_ns0\_reverted & -0.00 &    -0.2 &  0.857 &   [-0.0, 0.0] \\
\bottomrule
\end{tabular}

\end{center}
\caption{m2\_days\_active \texttildelow\ m1\_days\_active + m1\_received\_harassment + m1\_warnings +  m1\_fraction\_ns0\_deleted + m1\_fraction\_ns0\_reverted }
\label{tab:}
\end{table}
    


,coef,z-stat,p-val,95% CI
,,,,
Intercept,-0.60,-49.8,0.000,"[-0.6, -0.6]"
m1_days_active,0.51,158.9,0.000,"[0.5, 0.5]"
m1_received_harassment,-0.53,-13.0,0.000,"[-0.6, -0.5]"
m1_warnings,-0.12,-7.7,0.000,"[-0.2, -0.1]"
m1_fraction_ns0_deleted,-0.05,-1.1,0.272,"[-0.1, 0.0]"
m1_fraction_ns0_reverted,-0.00,-0.2,0.857,"[-0.0, 0.0]"


WIP: Receiving harassment is worse for a newcomer than receiving 11 warning messages or having all their first months work deleted or reverted.

### Investigate some newcomer experiences

Our regression analyses have established that newcomer who receive harassment show a greater subsequent decline in activity than normal. Let's look at a few example of newcomers, what edits they made and how other interacted with them.

In [7]:
cols = ['user_text', 'registration_day', 't1_harassment_received', 't1_harassment_made', 't1_num_days_active', 't2_num_days_active']
df_uxr = df_reg[cols]

In [ ]:
def find_attacks_made(u):
    

def find_attacks_received():